In [124]:
import spacetrack.operators as op
from spacetrack import SpaceTrackClient
import pandas as pd
import skyfield.api as sky
from skyfield import timelib
import pymongo
import time
import json

st = SpaceTrackClient(identity='alexandreabouchahine@gmail.com', password='a2SzJcdBiqtd9j!')

In [125]:
mc = pymongo.MongoClient(username="alex", password="alex", connect=True)

In [128]:
sat = st.satcat(orderby='launch')

In [129]:
ls = st.generic_request("launch_site")
ls = {
    x["SITE_CODE"]: x["LAUNCH_SITE"] 
    for x in ls
}

In [130]:
countries = st.boxscore()
countries_mapping = {
    x["SPADOC_CD"]: x["COUNTRY"]
    for x in countries
}

In [131]:
remove_keys = {
    'COMMENT', 'COMMENTCODE', 'CURRENT', 'FILE', 'INTLDES', 'LAUNCH_NUM', 'LAUNCH_PIECE',
    'LAUNCH_YEAR', 'OBJECT_NAME', 'RCSVALUE', 'RCS_SIZE'
}
for s in sat:
    s["SITE"] = ls.get(s["SITE"], s["SITE"])
    s["COUNTRY"] = countries_mapping.get(s["COUNTRY"], s["COUNTRY"])
    for rk in remove_keys:
        del s[rk]

In [132]:
mc["sat"].catalogue.insert_many(sat)
mc["sat"].countries.insert_many(countries)

In [133]:
with open("satellites.txt", "w") as f:
    f.write(st.gp(epoch='>now-10', decay_date=None, format='3le'))

In [138]:
# satellites = sky.load.tle_file("satellites.txt", skip_names=False)
# active_sat = []
# ts = sky.load.timescale()
# t = ts.now() 
# for s in satellites:
#     lat = s.at(t).subpoint().latitude.degrees
#     long = s.at(t).subpoint().longitude.degrees
#     if s.name=="TBA - TO BE ASSIGNED" or np.isnan(lat) or np.isnan(long):
#         continue
#     active_sat.append({
#         'id': s.model.satnum,
#         'name': s.name,
#         'latitude': lat,
#         'longitude': long,
#         }
#     )

with open("satellites.txt", "w") as f:
    f.write(st.gp(epoch='>now-10', decay_date=None, format='tle'))

with open("satellites.txt", "r") as f:
    a = f.readlines()
    
active_sat = []
i = 0 
while i < len(a):
    s = {
        "line1": a[i].rstrip(),
        "line2": a[i+1].rstrip()
        }
    active_sat.append(s)
    i += 2
    
mc["sat"].active.insert_many(active_sat)

In [139]:
with open("french_sats.json", "r") as f:
    d = json.load(f)

In [140]:
mc["sat"].active_french.insert_many(d)

In [142]:
files = [
    ("geo_lakes", "geodata/ne_50m_lakes.geojson"),
    ("geo_rivers", "geodata/ne_110m_rivers_lake_centerlines.geojson"),
    ("geo_countries", "geodata/ne_110m_admin_0_countries.geojson"),
    ("geo_oceans", "geodata/oceans.json"),
]

for name, file in files:
    with open(file, "r") as f:
        data = json.load(f)
    mc["sat"][name].insert_one(data)